In [43]:

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.getAll()

#sc = SparkContext()
#sc.stop()

def main():
    spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

    df = spark.read.option("header", True).csv("../data/conditions_truncated.csv")
    df.printSchema()

    df.show(n=10, truncate=False)

if __name__ == "__main__":
    main()

root
 |-- START: string (nullable = true)
 |-- STOP: string (nullable = true)
 |-- PATIENT: string (nullable = true)
 |-- ENCOUNTER: string (nullable = true)
 |-- CODE: string (nullable = true)
 |-- DESCRIPTION: string (nullable = true)

        START        STOP                               PATIENT  \
0  2017-01-14  2017-03-30  09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a   
1  2012-09-15  2012-09-16  b0a03e8c-8d0f-4242-9548-40f4d294eba8   
2  2018-06-17  2018-06-24  09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a   
3  2019-04-19  2019-09-26  09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a   
4  2019-04-27  2019-05-18  09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a   
5  2019-06-03  2019-08-02  09e4e8cb-29c2-4ef4-86c0-a6ff0ba25d2a   
6  2014-11-09  2014-11-30  b0a03e8c-8d0f-4242-9548-40f4d294eba8   
7  2015-01-04  2015-01-18  b0a03e8c-8d0f-4242-9548-40f4d294eba8   
8  2015-03-26        None  b0a03e8c-8d0f-4242-9548-40f4d294eba8   
9  2015-08-04  2015-08-14  b0a03e8c-8d0f-4242-9548-40f4d294eba8   

                        